# Paso 6: Construye el modelo y optimízalo

### Elección del Modelo y Entrenamiento: Dada la naturaleza de clasificación, hemos decidido explorar los modelos de Regresión Logística, Random Forest y XGBoost

In [ ]:
import pandas as pd

train_data = pd.read_csv("/workspaces/Proyecto_Final_Close-Loop-Intelligence/data/processed/X_train_selected.csv")
test_data = pd.read_csv("/workspaces/Proyecto_Final_Close-Loop-Intelligence/data/processed/X_test_selected.csv")

train_data.head()

In [ ]:
train_data.info()

In [ ]:
# Dividimos los datos en características (X) y variable objetivo (y)

X_train = train_data.drop(columns=['ClasificacionABC'])
y_train = train_data['ClasificacionABC']
X_test = test_data.drop(columns=['ClasificacionABC'])
y_test = test_data['ClasificacionABC']

In [ ]:
# 1. Ver qué hay realmente en la columna
print(train_data['ClasificacionABC'].unique())

### 1) Modelo de Regresión Logística

In [1]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Predicciones y evaluación del modelo

y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(cm)

El modelo predice solo la clase 0, entendemos que se debe al desbalance de clases.

In [ ]:
model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)

In [ ]:
# Predicciones y evaluación del modelo

y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(cm)

### 2) Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf_model = RandomForestClassifier(random_state = 42,class_weight='balanced')
rf_model.fit(X_train, y_train)

In [ ]:
# Predicciones y evaluación del modelo

y_pred = rf_model.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(cm)

In [ ]:
hyperparameters = {
    'n_estimators': [50, 100],
    'max_depth': [10, 20, None],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# Optimización del modelo

grid_search = GridSearchCV (rf_model, param_grid=hyperparameters, cv=3, scoring='f1_weighted', n_jobs=-1)
grid_search

In [ ]:
# 1. Ajustar el modelo con los datos
grid_search.fit(X_train, y_train)

# 2. Obtener el mejor modelo
best_rf = grid_search.best_estimator_
print(f"Mejores parámetros: {grid_search.best_params_}")

In [ ]:
# 3. Predicciones y Evaluación
y_pred_rf = best_rf.predict(X_test)

print("\n--- Reporte de Clasificación (Random Forest Optimizado) ---")
report_rf_optimized = classification_report(y_test, y_pred_rf)
accuracy_rf_optimized = accuracy_score(y_test, y_pred_rf)
print(report_rf_optimized)

print("\n--- Matriz de Confusión ---")
print(confusion_matrix(y_test, y_pred_rf))

### 3) XGBoost

In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)

In [ ]:
# Predicciones y evaluación del modelo

y_pred = xgb_model.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(cm)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Optimización del modelo

hyperparameters = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 9],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

random_search_xgb = RandomizedSearchCV(xgb_model, hyperparameters, n_iter=10, cv=3, scoring='f1_weighted', n_jobs=-1)

# 1. Ajustar el modelo con los datos
random_search_xgb.fit(X_train, y_train)

# 2. Obtener el mejor modelo
best_xgb = random_search_xgb.best_estimator_
print(f"Mejores parámetros: {random_search_xgb.best_params_}")

# 3. Predicciones y Evaluación
y_pred_xgb_optimized = best_xgb.predict(X_test)

print("--- Reporte de Clasificación (XGBoost Optimizado) ---")
report_xgb_optimized = classification_report(y_test, y_pred_xgb_optimized)
accuracy_xgb_optimized = accuracy_score(y_test, y_pred_xgb_optimized)
print(report_xgb_optimized)

print("")
print("--- Matriz de Confusión ---")
print(confusion_matrix(y_test, y_pred_xgb_optimized))

**Conclusión General:**

Ambos modelos lograron un accuracy similar. Sin embargo, considerando la naturaleza del problema de clasificación (con un desbalance de clases significativo), el **Random Forest optimizado** demuestra ser un modelo más equilibrado. Aunque su recall en las clases minoritarias no es extremadamente alto, es sustancialmente mejor que el de XGBoost, lo que lo hace más útil para identificar las instancias de las clases 1 y 2, que probablemente son de mayor interés para el negocio.

In [ ]:
# Guardado del modelo

from pickle import dump

dump(best_rf, open('/workspaces/Proyecto_Final_Close-Loop-Intelligence/models/best_rf_model.sav', 'wb'))